### 해보고 싶은거:
- 그레이 필터에서 원하는 색상만 가져오는 코드
- → 복제로 가야겠다..
- RGB 표시하는 거 가져와서
- 해당 부분의 색깔만 남기고 나머지는 회색처리..
- 슬라이싱으로 해당 부분만 제외할 수 있나?

### 회색 이미지는 2차원, 3차원 회색이미지로 만들고 싶으면 bgr2gray 후 다시 gray2bgr 하기!

In [1]:
import cv2
import numpy as np

In [2]:
img = cv2.imread('../textbook/img/cube.jpg')      # 이미지를 로드합니다.
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 이미지를 그레이스케일로 변환합니다.

# 그레이스케일 이미지를 3채널 회색 이미지로 변환합니다.
gray_img_3channel = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)

In [5]:
# 기본 이미지는 3차원, 채널이 세 개
img.shape

(740, 756, 3)

In [4]:
# 회색 이미지는 2차원
gray_img.shape

(740, 756)

# 1. 그레이 이미지에 크롭하면 새창으로 원본 색상 띄우기

In [29]:
isDragging = False                      # 마우스 드래그 상태 저장

x0, y0, w, h = -1,-1,-1,-1              # 영역 선택 좌표 저장
blue, red = (255,0,0),(0,0,255)         # 색상 값


# 함수 선언할 때 전역변수 필요하면 꼭 global 선언해주기
def onMouse(event,x,y,flags,param):     # 마우스 이벤트 핸들 함수
    global isDragging, x0, y0, img, gray_img_3channel      # 전역변수 참조
    
    if event == cv2.EVENT_LBUTTONDOWN:  # 왼쪽 마우스 버튼 다운, 드래그 시작 
        isDragging = True
        x0 = x
        y0 = y
    
    elif event == cv2.EVENT_MOUSEMOVE:  # 마우스 움직임
        if isDragging:                  # 드래그 진행 중
            img_draw = img.copy()       # 사각형 그림 표현을 위한 이미지 복제
            cv2.rectangle(img_draw, (x0, y0), (x, y), blue, 2) # 드래그 진행 영역 표시
            cv2.imshow('img', img_draw) # 사각형 표시된 그림 화면 출력
    
    elif event == cv2.EVENT_LBUTTONUP:  # 왼쪽 마우스 버튼 업
        if isDragging:                  # 드래그 중지
            isDragging = False
            w = x - x0                  # 드래그 영역 폭 계산
            h = y - y0                  # 드래그 영역 높이 계산
            print("x:%d, y:%d, w:%d, h:%d" % (x0, y0, w, h))
            
            if w > 0 and h > 0:         # 폭과 높이가 음수이면 드래그 방향이 옳음 
                img_draw = img.copy()   # 선택 영역에 사각형 그림을 표시할 이미지 복제
                # 선택 영역에 빨간 사각형 표시
                cv2.rectangle(img_draw, (x0, y0), (x, y), red, 2)
                cv2.imshow('img', gray_img_3channel) # 빨간 사각형 그려진 이미지 화면 출력
                roi = img[y0:y0+h, x0:x0+w] # 원본 이미지에서 선택 영영만 ROI로 지정 
                cv2.imshow('cropped', roi)  # ROI 지정 영역을 새창으로 표시
                cv2.moveWindow('cropped', 0, 0) # 새창을 화면 좌측 상단에 이동
                cv2.imwrite('./cropped.jpg', roi)   # ROI 영역만 파일로 저장 
                print("croped.")
                
            else:
                cv2.imshow('img', gray_img_3channel)  # 드래그 방향이 잘못된 경우 사각형 그림이 없는 원본 이미지 출력
                print("좌측 상단에서 우측 하단으로 영역을 드래그 하세요.")
            

In [32]:
img = cv2.imread('../textbook/img/cube.jpg')
cv2.imshow('img', gray_img_3channel)
cv2.setMouseCallback('img', onMouse) # 마우스 이벤트 등록
cv2.waitKey()
cv2.destroyAllWindows()

x:262, y:248, w:243, h:227
croped.


- 완벽하게는 아니지만 어느정도 됐당.
- 근데 원본 이미지의 붉은색 부분 테두리가 아예 흰색으로 처리된다 → 코드가 잘못 되었던 부분이 있었던 듯. 23일엔 멀쩡함.
- 오 근데 이걸 좀 변형하면 해당 부분만 색깔있게 표시할 수 있겠다.

# 2. 크롭한 부분 색상만 원본 색상으로 비추기

In [10]:
isDragging = False                      # 마우스 드래그 상태 저장

x0, y0, w, h = -1,-1,-1,-1              # 영역 선택 좌표 저장
blue, red = (255,0,0),(0,0,255)         # 색상 값


# 함수 선언할 때 전역변수 필요하면 꼭 global 선언해주기
def onMouse(event,x,y,flags,param):     # 마우스 이벤트 핸들 함수
    global isDragging, x0, y0, img, gray_img_3channel      # 전역변수 참조
    
    gray_img = gray_img_3channel.copy()
    if event == cv2.EVENT_LBUTTONDOWN:  # 왼쪽 마우스 버튼 다운, 드래그 시작
        isDragging = True
        x0 = x
        y0 = y
    
    elif event == cv2.EVENT_MOUSEMOVE:  # 마우스 움직임 
        if isDragging:                  # 드래그 진행 중
            img_draw = img.copy()       # 사각형 그림 표현을 위한 이미지 복제
            cv2.rectangle(img_draw, (x0, y0), (x, y), blue, 2) # 드래그 진행 영역 표시
            cv2.imshow('img', img_draw) # 사각형 표시된 그림 화면 출력
    
    elif event == cv2.EVENT_LBUTTONUP:  # 왼쪽 마우스 버튼 업 
        if isDragging:                  # 드래그 중지
            isDragging = False
            w = x - x0                  # 드래그 영역 폭 계산
            h = y - y0                  # 드래그 영역 높이 계산
            print("x:%d, y:%d, w:%d, h:%d" % (x0, y0, w, h))
            
            if w > 0 and h > 0:         # 폭과 높이가 음수이면 드래그 방향이 옳음 
                img_draw = img.copy()    # 선택 영역에 사각형 그림을 표시할 이미지 복제
                cv2.imshow('img', gray_img) # 빨간 사각형 그려진 이미지 화면 출력
                roi = img[y0:y0+h, x0:x0+w] # 원본 이미지에서 선택 영역만 ROI로 지정 
                gray_img[y0:y0+h, x0:x0+w] = roi
                cv2.imshow('img', gray_img)
                
            else:
                cv2.imshow('img', gray_img)  # 드래그 방향이 잘못된 경우 사각형 그림이 없는 원본 이미지 출력
                print("좌측 상단에서 우측 하단으로 영역을 드래그 하세요.")
            

In [11]:
img = cv2.imread('../textbook/img/cube.jpg')
cv2.imshow('img', gray_img_3channel)
cv2.setMouseCallback('img', onMouse) # 마우스 이벤트 등록 
cv2.waitKey()
cv2.destroyAllWindows()

x:248, y:158, w:415, h:325
x:129, y:204, w:239, h:296
x:374, y:269, w:303, h:263
x:372, y:242, w:182, h:257
